In [6]:
%matplotlib inline
import torch
from sebm.sgld import SGLD_sampler
from sebm.models import CEBM_GMM_2ss
dataset =  'svhn' # 'svhn' # 'cifar10' # 'mnist' #  'flowers102' #
if dataset == 'mnist' or dataset =='fashionmnist':
    input_channels, im_height, im_width = 1, 28, 28
else:
    input_channels, im_height, im_width = 3, 32, 32
device = torch.device('cuda:0')
arch =  'simplenet2' 
ss = 2
seed =  3
# lr, reg_alpha = 1e-4, 1e-1 # 
lr ,reg_alpha = 5e-5, 1e-1

optimize_priors = True
num_clusters = 10
if dataset == 'cifar10' or dataset == 'svhn':
    channels, kernels, strides, paddings =[64,128,256,512], [3,4,4,4], [1,2,2,2], [1,1,1,1]
    hidden_dim, latent_dim, activation = [1024], 128, 'Swish'
elif dataset == 'mnist' or dataset == 'fashionmnist':
    
    channels, kernels, strides, paddings =[64,64,32,32], [3,4,4,4], [1,2,2,2], [1,1,1,1]
    hidden_dim, latent_dim, activation = [128], 128, 'Swish'
else:
    raise NotImplementError

data_noise_std = 3e-2
sgld_noise_std, sgld_lr, sgld_num_steps = 7.5e-3, 2.0, 60
buffer_init, buffer_dup_allowed = True, True
data_dir = '../../../sebm_data/'
load_version = 'cebm_gmm_k=%d-d=%s-seed=%d-lr=%s-zd=%d-d_ns=%s-sgld-ns=%s-lr=%s-steps=%s-reg=%s-act=%s-arch=%s' % (num_clusters, dataset, seed, lr, latent_dim, data_noise_std, sgld_noise_std, sgld_lr, sgld_num_steps,  reg_alpha, activation, arch)
if arch == 'simplenet2':
    ebm = CEBM_GMM_2ss(K=num_clusters,
                    arch=arch,
                    optimize_priors=optimize_priors,
                    device=device,
                    im_height=im_height, 
                    im_width=im_width, 
                    input_channels=input_channels, 
                    channels=channels, 
                    kernels=kernels, 
                    strides=strides, 
                    paddings=paddings, 
                    hidden_dim=hidden_dim,
                    latent_dim=latent_dim,
                    activation=activation)
else:
    raise NotImplementError
ebm = ebm.cuda().to(device)
print('Loading trained weights..')
weights = torch.load('../weights/weights/1012/cp-%s' % load_version)
ebm.load_state_dict(weights['model_state_dict'])
# if not optimize_priors:
#     ebm.prior_mu = weights['prior_mu'].to(device)
#     ebm.prior_log_sigma = weights['prior_log_sigma'].to(device)

Loading trained weights..


<All keys matched successfully>

In [7]:
from sebm.eval import *
evaluator = Evaluator_EBM_GMM(ebm, device, dataset, data_dir, data_noise_std=1e-2)

In [8]:
fewshots(model_name='cebm_gmm', evaluator=evaluator, list_num_shots=[1])

  0%|          | 0/1 [00:00<?, ?it/s]

Using downloaded and verified file: ../../../sebm_data/SVHN/test_32x32.mat
Using downloaded and verified file: ../../../sebm_data/SVHN/test_32x32.mat
Using downloaded and verified file: ../../../sebm_data/SVHN/test_32x32.mat
Using downloaded and verified file: ../../../sebm_data/SVHN/test_32x32.mat
Using downloaded and verified file: ../../../sebm_data/SVHN/test_32x32.mat
Using downloaded and verified file: ../../../sebm_data/SVHN/test_32x32.mat
Using downloaded and verified file: ../../../sebm_data/SVHN/test_32x32.mat
Using downloaded and verified file: ../../../sebm_data/SVHN/test_32x32.mat
Using downloaded and verified file: ../../../sebm_data/SVHN/test_32x32.mat
Using downloaded and verified file: ../../../sebm_data/SVHN/test_32x32.mat


100%|██████████| 1/1 [01:25<00:00, 85.53s/it]


In [ ]:
metrics =  ['pixel','vae_z', 'cebm_z', 'igebm_z']
datasets = ['mnist', 'cifar10']
import torch
import numpy as np
import matplotlib.pyplot as plt
for d in datasets:
    for m in metrics:
        plot_confusion_matrix(d, m)

In [ ]:
def plot_confusion_matrix(dataset, metric):
    l = torch.load('confusion/logging/confusion_matrix_labels_%s_%s.pt' % (metric, dataset)).long()
    cm = torch.zeros((10, 10))
    for i in range(len(l)):
        cm[l[i, 0], l[i, 1]] += 1
    for j in range(len(cm)):
        cm[j] /= cm[j].sum()
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    im = ax.imshow(cm, cmap='inferno', vmax=1.0, vmin=0.0)
    if metric == 'pixel':
        model = 'Pixel'
    elif metric == 'vae_z':
        model = 'VAE'
    elif metric == 'igebm_z':
        model = 'IGEBM'
    elif metric == 'cebm_z':
        model = 'CEBM'
    else:
        raise NotImplementError
    ax.set_title('%s' % model, fontsize=20)
#     plt.colorbar(im)
    ticks = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    ax.set_xticks(np.arange(len(ticks)))
    ax.set_yticks(np.arange(len(ticks)))
    ax.set_xticklabels(ticks)
    ax.set_yticklabels(ticks)
    ax.set_ylabel('True Class Labels', fontsize=20)
    ax.set_xlabel('Predicted Class Labels', fontsize=20)
    for i in range(len(ticks)):
        for j in range(len(ticks)):
            text = ax.text(j, i, round(cm[i, j].item(), 2),
                           ha="center", va="center", color=("k" if i == j else 'w'))
    plt.savefig('confusion/figures/cm_%s_%s.png' % (dataset, metric))

In [ ]:
datasets = ['svhn', 'texture', 'constant_rgb']
scores = ['energy', 'gradient']
for d in datasets:
    for s in scores:
        auroc = evaluator.oodauc(dataset_ood=d, score=s)
        print('dataset=%s, score=%s, auroc=%.3f' % (d,s,auroc))

In [ ]:
# logging_interval = 200 # None otherwise
# test_batch_size = 10
# init_samples, labels = draw_one_batch(1, dataset, data_dir, train=False, normalize=True, flatten=False)

# images_ebm = evaluator.uncond_sampling_ll(sample_size=3,
#                                           batch_size=test_batch_size, 
#                                           sgld_steps=2000,
#                                           sgld_lr=2.0,
#                                           sgld_noise_std=1e-3,
#                                           init_images=init_samples,
#                                           grad_clipping=False,
#                                           logging_interval=None)
# nearestneighbours = evaluator.nn_latents(images_ebm)
# plot_evolving_samples(images_ebm, nearestneighbours, fs=8, save_name=None)

In [ ]:
# evaluator.plot_all_samples(list_images=images_ebm, fs=1.0)

In [ ]:
images_ebm = evaluator.uncond_sampling(batch_size=100, 
                                          sgld_steps=1000,
                                          sgld_lr=2.0,
                                          sgld_noise_std=1e-3,
                                          grad_clipping=False,
                                          init_samples=None,
                                          logging_interval=None)
evaluator.plot_final_samples(images_ebm, fs=5, save=False)

In [ ]:
# evaluator.plot_all_samples(images_ebm, fs=6, save=True)

In [ ]:
evaluator.plot_oods(dataset_ood='cifar10', score='gradient', train=False, fs=4, save=False)

In [ ]:
# train_logistic_classifier(evaluator)

In [ ]:
zs, ys = evaluator.compute_tsne()
evaluator.plot_tsne(zs, ys, save=True)